In [1]:
import os 
import codecs
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
np.random.seed(7)
#Read The TSV Data File
set_num  = 1
data = pd.read_table('../data/train_rel_2.tsv')
data = data[data["EssaySet"] == set_num]
essay_score = np.asarray(data['Score1'].tolist())
essay_set = np.asarray(data['EssaySet'].tolist())
essay_text = np.asarray(data["EssayText"].tolist())

FileNotFoundError: File b'../data/train_rel_2.tsv' does not exist

In [138]:
len(essay_score)

1672

In [139]:
import numpy as np

feature_1 = np.array([1]*len(np.load("context_feature.npy"))-(np.load("context_feature.npy")))
feature_2 = np.load("pos_tags_overlap_features.npy")
feature_3 = np.array([1]*len(np.load("context_feature.npy"))-np.load("bert_feature.npy"))


In [140]:
## set wise feature extraction

whole_data = pd.read_table('../data/train_rel_2.tsv')
whole_data = whole_data["EssaySet"].tolist()
start = whole_data.index(set_num)
end = len(whole_data) - whole_data[::-1].index(set_num)

feature_1 = feature_1[start:end]
feature_2 = feature_2[start:end]
feature_3 = feature_3[start:end]

In [141]:
indices = np.arange(data.shape[0])
print(indices)
# np.random.shuffle(indices)

labels = essay_score[indices]
# print(labels[1])
essay_set = essay_set[indices]
essay_text = essay_text[indices]

feature_1 = feature_1[indices]
feature_2 = feature_2[indices]
print(len(feature_2))
feature_3 = feature_3[indices]
print(len(feature_3))

[   0    1    2 ... 1669 1670 1671]
1672
1672


In [142]:
#Size for train-test split
train_size = int(0.7 * len(data))

# set_train = essay_set[:train_size]
# set_test = essay_set[train_size:]
# print(set_train)
feature_1_train = feature_1[:train_size]
feature_1_test = feature_1[train_size:]

feature_2_train = feature_2[:train_size]
feature_2_test = feature_2[train_size:]

feature_3_train = feature_3[:train_size]
feature_3_test = feature_3[train_size:]

features_train = np.column_stack((feature_1_train, feature_2_train, feature_3_train))
features_test = np.column_stack((feature_1_test, feature_2_test, feature_3_test))
# features_test = np.column_stack((set_test,sent_count_test,word_count_test,avg_word_len_test,count_noun_test,count_adj_test,count_verb_test,count_adv_test,noun_overlap_test,adj_overlap_test,verb_overlap_test,adv_overlap_test,count_spell_error_test,count_lemmas_test,polarity_score_test,rubrik_cosine_test,lda_test[:,0],lda_test[:,1],lda_test[:,2],lda_test[:,3],lda_test[:,4],lda_test[:,5],lda_test[:,6],lda_test[:,7],lda_test[:,8],lda_test[:,9], question_cosine_test, google_score_test[:,0], google_score_test[:,1], google_score_test[:,2], google_score_test[:,3], google_score_test[:,4], google_score_test[:,5], google_score_test[:,6], google_score_test[:,7], google_score_test[:,8], google_score_test[:,9], perplexity_score_test, readability_score_test, subjectivity_score_test, rubrik_cosine_doc2vec_test, rubrik_min_max_bert_test, question_cosine_min_max_bert_test, ari_score_test, dc_read_test, grade_level_test, gunning_fog_test, keywords_overlap_test, liau_index_test, reading_ease_test, smog_index_test, complex_word_test, lix_score_test, rix_score_test, type_token_test, unique_count_test, connective_add_test, connective_casual_test, connective_inci_test, connective_logic_test, connective_temporal_test, content_diversity_test, lexical_diversity_test, logic_and_test, logic_if_test, logic_neg_test, logic_operator_test, logic_or_test, n_gram_test))

y_train = labels[:train_size]
y_test = labels[train_size:]

# print(X_train.shape)
print(features_train.shape)
print(features_test.shape)

(1170, 3)
(502, 3)


In [143]:
from sklearn.ensemble import ExtraTreesClassifier
import pandas
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import RFE

estimators = []
num_trees = 10
model1 = ExtraTreesClassifier(n_estimators=num_trees, random_state=7,min_samples_leaf=1, min_samples_split=9)
selector1 = RFE(model1, 61, step=1)
estimators.append(('gb1', selector1))
num_trees = 30
model2 = GradientBoostingClassifier(n_estimators=num_trees,random_state=7,learning_rate=0.9,max_depth=3,min_samples_leaf=1,min_samples_split=2)
selector2 = RFE(model2, 60, step=1)
estimators.append(('gb2', selector2))
model3 = RandomForestClassifier(n_estimators=15)
selector3 = RFE(model3, 60, step=2)
estimators.append(('gb3', selector3))

# create the ensemble model
ensemble = VotingClassifier(estimators)
selector = RFE(ensemble, 56, step=2)

selector = ensemble.fit(features_train, y_train)

In [144]:
y_pred = selector.predict(features_test)

In [145]:
# !pip install ml_metrics
import ml_metrics as metrics
print("kappa:", metrics.quadratic_weighted_kappa(y_pred,y_test))

kappa: 0.2634186906381868


In [136]:
0.33,0.25,0.07

(0.33, 0.25, 0.07)